In [1]:
print("test")

test


In [5]:
import re
sample_file_name = "../dutch-language/data/sample-sentences-nlwiki.txt"
short_file_name = "../dutch-language/data/short-sentences-nlwiki.txt"
full_file_name = "../dutch-language/data/clean-sentences-nlwiki.txt"
corrected_file_name = "../dutch-language/data/corrected-sentences-nlwiki.txt"

file_name = corrected_file_name

regex_worden = re.compile(r'(.+ z|Z)end\b.*')
with open(file_name) as file:
    total_count = 0
    match_count = 0

    for line in file:
        if (total_count >= 1000000):
            break
        total_count += 1
        clean_line = line.rstrip()
        if (regex_worden.match(clean_line)):
            match_count += 1
            if (total_count > 500000):
                print(clean_line)

    print(total_count)
    print(match_count)

Voor communicatie werd het zend- en ontvangstation Radio Malabar gebruikt, dat intercontinentaal telefoneren mogelijk maakte met Nederland.
PZB bestaat uit een passieve LC-kring naast het spoor bij het sein en een actieve zend/ontvangspoel aan de trein.
in Openbaringen 1 schrijft Johannes het volgende: Hetgeen gij ziet, schrijf dat in een boek en zend het aan de zeven gemeenten.
De zend- en ontvangstapparatuur is meestal ondergebracht in een gebouw, kast of container.
RTL 4 zend het programma in december 2019 en januari 2020 uit.
Een oplossing is dan een digitale hoofdtelefoon die in en rond de 2,4 GHz ontvangt en zend.
En in het heldendicht Hákonarmál is het Hákon de Goede die naar Walhalla wordt gevoerd door de walkure Göndul en Odin zend Hermóðr en Bragi om hem te begroeten.
Val binnen als de bewoners om hulp vragen, neem het land in, zend kolonisten naar het gebied of de vorst moet er zelf gaan wonen.
1000000
21


In [5]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel(
    "bert", "outputs/BERT-001", use_cuda=True
)
print(type(model))


<class 'simpletransformers.classification.classification_model.ClassificationModel'>


In [6]:
predictions, raw_outputs = model.predict([
    "Ik word nieuwsgierig.",
    "Hoe word je gevraagd?",
    "Wat wordt je gevraagd?",
    "Wat word je opdringerig, zeg!",

    "Ik zend een pakje.",
    "Ik zend je een pakje.",
    "Ik zend u een heel mooie trui.",
    "Je zendt het toch nog vandaag op, hoop ik.", #
    "Jij zendt een hele grote doos naar oma.",
    "Zend je dat straks uit?", #
    "Zend jij echt die fiets via de post?",
    "Welke baas zendt je nu helemaal naar London voor 1 klant?",

    # With spelling mistake
    "Ik wordt nieuwsgierig.",
    "Hoe wordt je gevraagd?",
    "Wat word je gevraagd?",
    "Wat wordt je opdringerig, zeg!",

    "Ik zendt een pakje.",
    "Ik zendt je een pakje.",
    "Ik zendt u een heel mooie trui.", #
    "Je zend het toch nog vandaag op, hoop ik.",
    "Jij zend een hele grote doos naar oma.",
    "Zendt je dat straks uit?", #
    "Zendt jij echt die fiets via de post?",
    "Welke baas zend je nu helemaal naar London voor 1 klant?",

    # Trying other words
    # Correct spelling
    "Ik vind dit toch niet zo mooi.",
    "Wat vind jij van al die aandacht?",
    "Hoe vind ik nu de ingang?",
    "Het is toch erg dat hij dat niet vindt.",
    "Hoe zwaar vindt hij de opleiding?",

    "Ik loop er zo maar voorbij.",
    "Loop jij ook zo snel?",
    "Je loopt daar beter niet telkens over.",
    "Jij loopt echt helemaal naar zee?",
    "En daarom loopt hij er met een grote bocht omheen.",

    # With spelling mistake
    "Ik vindt dit toch niet zo mooi.",
    "Wat vindt jij van al die aandacht?",
    "Hoe vindt ik nu de ingang?",
    "Het is toch erg dat hij dat niet vind.", # Incorrectly evaluated to "0"
    "Hoe zwaar vind hij de opleiding?",

    "Ik loopt er zo maar voorbij.",
    "Loopt jij ook zo snel?",
    "Je loop daar beter niet telkens over.",
    "Jij loop echt helemaal naar zee?",
    "En daarom loop hij er met een grote bocht omheen."
])
print()
print(predictions)
print(raw_outputs)

# Expected           [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
# Actual RoBERTa-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1]
# Actual RoBERTa-002 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 1 1]
# Actual    BERT-001 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0]



INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
100%|██████████| 6/6 [00:01<00:00,  4.50it/s]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1
 0 0 1 1 0 1 0]
[[ 4.9011836 -4.9474177]
 [ 4.7910857 -4.720821 ]
 [ 4.733791  -4.921354 ]
 [ 4.9177537 -4.913611 ]
 [ 4.838971  -4.778938 ]
 [ 4.9278646 -4.7669377]
 [ 4.855145  -4.727303 ]
 [ 4.7080703 -4.9509273]
 [ 4.7960134 -5.075569 ]
 [ 4.794192  -4.6371403]
 [ 4.8583226 -4.9751067]
 [ 4.6117396 -5.0969477]
 [-5.32149    4.9963202]
 [-4.9000816  4.544892 ]
 [-5.1790657  4.9842153]
 [-5.2553926  4.8809614]
 [-5.3517423  4.946244 ]
 [-5.306557   4.949043 ]
 [-5.3236747  4.942291 ]
 [-5.039869   4.9698987]
 [-5.159084   4.982388 ]
 [-5.280131   4.984758 ]
 [-5.343091   4.9652734]
 [-5.1575484  4.978852 ]
 [ 3.7812762 -4.1465325]
 [ 4.730141  -4.9689956]
 [ 4.1573477 -4.5523114]
 [ 4.223512  -4.4708   ]
 [ 4.752958  -5.067515 ]
 [ 4.5089774 -4.879441 ]
 